In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 02:08:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/03 02:08:41 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
df_green = spark.read.parquet("data/green/*/*")

In [4]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-23 13:10:15|  2020-01-23 13:38:16|                 N|         1|          74|         130|              1|        12.77|       36.0|  0.0|    0.

In [5]:
df_green.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [7]:
df_green.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [8]:
df_green.count()

2304517

In [9]:
df_green.registerTempTable('green')

/home/rizky/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [23]:
spark.sql("""
SELECT VendorID, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID
FROM green
LIMIT 10;
""").show()

+--------+-------------------+-------------------+------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+--------+-------------------+-------------------+------------+------------+
|       2|2020-01-23 13:10:15|2020-01-23 13:38:16|          74|         130|
|       2|2020-01-15 16:56:36|2020-01-15 17:19:05|          97|          33|
|       1|2020-01-16 05:55:12|2020-01-16 06:30:08|          82|          68|
|       2|2020-01-14 12:50:42|2020-01-14 12:55:46|          75|          74|
|       2|2020-01-07 06:36:27|2020-01-07 06:48:27|         244|          74|
|       2|2020-01-04 10:28:49|2020-01-04 10:32:45|          42|          41|
|       2|2020-01-19 12:30:08|2020-01-19 12:40:53|          74|         166|
|       1|2020-01-04 20:44:28|2020-01-04 20:52:24|         129|         129|
|       2|2020-01-21 23:13:47|2020-01-21 23:17:31|          61|          61|
|       2|2020-01-12 11:53:00|2020-01-12 11:57:33|           7|         179|

In [12]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', pickup_datetime) AS hour_month,
    PULocationID AS revenue_zone,
    SUM(total_amount) AS revenue_monthly_total_amount,
    COUNT(1) AS number_records
FROM
    green
GROUP BY
    1, 2
ORDER BY
    1, 2
""").show()

+-------------------+------------+----------------------------+--------------+
|         hour_month|revenue_zone|revenue_monthly_total_amount|number_records|
+-------------------+------------+----------------------------+--------------+
|2008-12-31 22:00:00|         193|                         0.0|             1|
|2008-12-31 23:00:00|         166|                        11.3|             1|
|2008-12-31 23:00:00|         193|                         0.0|             2|
|2008-12-31 23:00:00|         264|                         0.0|             1|
|2009-01-01 00:00:00|           7|                         9.8|             1|
|2009-01-01 00:00:00|          41|          15.850000000000001|             2|
|2009-01-01 00:00:00|          42|                         8.3|             1|
|2009-01-01 00:00:00|          74|                         9.8|             1|
|2009-01-01 00:00:00|          75|           98.39999999999999|             2|
|2009-01-01 00:00:00|         116|                  

In [13]:
df_zones_pd = pd.read_csv("taxi+_zone_lookup.csv")
df_zones_pd.dtypes

LocationID       int64
Borough         object
Zone            object
service_zone    object
dtype: object

In [16]:
df_zones_pd.head(5)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [17]:
zones_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [20]:
df_zones = spark.read \
    .option("header", "true") \
    .schema(zones_schema) \
    .csv("taxi+_zone_lookup.csv")

In [24]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [25]:
df_zones.registerTempTable('zones')

/home/rizky/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [28]:
df_green_revenue = spark.sql("""
SELECT
    *
FROM
    green AS g, zones AS z
WHERE
    g.PULocationID = z.LocationID
""").show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+----------+---------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|LocationID|  Borough|                Zone|service_zone|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+----------+---------+--------------------+------------+
|      